In [84]:
import numpy as np
import os
import pandas as pd
import torch
import albumentations as A
import albumentations.pytorch
import cv2
import sys
import random
import csv
import json
import matplotlib.pyplot as plt

import bisect
import copy
import math
from collections import defaultdict
from itertools import repeat, chain

import numpy as np
import torch
import torch.utils.data
import torchvision
from PIL import Image
from torch.utils.data.sampler import BatchSampler, Sampler
from torch.utils.data import Dataset

sys.path.append('../')
from visualize import visualize
from utils import *

%matplotlib inline

In [2]:
df = pd.read_csv('../saved/df.csv')
df.shape

(24782, 14)

In [10]:
df.head()

,file_name,task,label,xmin,ymin,w,h,occluded,des,cell_type,bbox,area,ratio,ID
0,patch_images/2021.01.14/LBC424-20210111(1)/LBC...,[AS6] LBC424,C,1539,199,139,211,0,NaN,ASC-US,"[1539, 199, 139, 211]",171,0.811645,0
1,patch_images/2021.01.14/LBC424-20210111(1)/LBC...,[AS6] LBC424,AS,1337,102,256,136,0,NaN,ASC-US,"[1337, 102, 256, 136]",186,1.371989,1
2,patch_images/2021.01.14/LBC424-20210111(1)/LBC...,[AS6] LBC424,AS,220,619,166,169,0,NaN,ASC-US,"[220, 619, 166, 169]",167,0.991085,2
3,patch_images/2021.01.14/LBC424-20210111(1)/LBC...,[AS6] LBC424,AS,658,1747,191,166,0,NaN,ASC-US,"[658, 1747, 191, 166]",178,1.072661,3
4,patch_images/2021.01.14/LBC424-20210111(1)/LBC...,[AS6] LBC424,AS,1571,365,136,146,0,NaN,ASC-US,"[1571, 365, 136, 146]",140,0.965146,4


In [3]:
class MyDataset:
    def __init__(self, df):
        self.df = df
    
    def __getitem__(self, i):
        return self.df.loc[i]['area']
    
    def __len__(self):
        return len(self.df)

In [4]:
dataset = MyDataset(df)

In [9]:
dataset[1]

186

In [113]:
areas = df.area
bins = [100, 140, 200, 300]

In [114]:
def _quantize(x, bins):
    bins = copy.deepcopy(bins)
    bins = sorted(bins)
    quantized = list(map(lambda y: bisect.bisect_right(bins, y), x))
    return quantized

In [115]:
groups = _quantize(areas, bins)

In [116]:
set(groups)

{0, 1, 2, 3, 4}

In [117]:
counts = np.unique(groups, return_counts=True)[1]
counts

array([2892, 5588, 8443, 5665, 2194])

In [60]:
fbins = [0] + bins + [np.inf]
fbins

[0, 120, 170, 250, inf]

In [32]:
areas = df.area
bins = [180]
groups = _quantize(areas, bins)
set(groups)

{0, 1}

In [33]:
counts = np.unique(groups, return_counts=True)[1]
counts

array([14541, 10241])

In [85]:
# https://stackoverflow.com/questions/68782144/pytorch-can-i-group-batches-by-length
class DS(Dataset):
    def __init__(self, files):
        super().__init__()
        self.len = len(files)
        self.files = files

    def __getitem__(self, index):
        return self.files[index]

    def __len__(self):
        return self.len

In [86]:
file_len = np.random.randint(0, 100, (16*6))
files = [np.random.rand(s) for s in file_len]
ds = DS(files)

In [92]:
ds[0]

array([0.2055477 , 0.57434555, 0.71370411, 0.85958713, 0.61770843,
       0.40486959, 0.78779201, 0.65046251, 0.51110209, 0.97244704,
       0.75316285, 0.21871491, 0.83272943, 0.61343948, 0.72190206,
       0.97705977, 0.74047199, 0.56858119, 0.29906154, 0.73553948,
       0.29797852, 0.75791858, 0.33054886, 0.62545127, 0.27807226,
       0.8301604 , 0.55101312, 0.88531032, 0.55043899, 0.9917991 ,
       0.59397518, 0.46668368, 0.20461569, 0.08638541, 0.87828026,
       0.9268614 , 0.21051897, 0.38086242, 0.42916876, 0.1750903 ,
       0.91178358, 0.33666994, 0.69793785, 0.18797078, 0.21452113,
       0.39479421, 0.22719111, 0.35459336, 0.56241222, 0.91490426,
       0.42528208, 0.31693724, 0.78183943, 0.66702575, 0.94409075])

In [104]:
file_len

array([55, 83, 94, 38, 55, 32, 25, 39, 39, 73, 45, 24, 90,  4, 47, 69, 71,
       65, 22, 78, 47, 61,  9, 77, 77, 41, 75, 39, 54, 23, 38, 38,  1, 59,
       22, 95, 61, 14, 62,  5, 70, 87,  7,  7, 60, 72, 73, 72, 18, 53, 74,
        6, 92, 97, 11, 36, 78, 49, 30, 66, 36,  1, 87, 73, 25, 82, 67, 61,
       29, 44, 80,  5, 80, 43, 93, 56, 24, 18, 31, 19, 47, 38, 29, 77, 81,
       96, 64, 91, 33, 51, 36, 44, 90, 19, 99, 91])

In [107]:
# sort and return sorted index
file_len.argsort()[::-1]

array([94, 53, 85, 35,  2, 74, 52, 87, 95, 12, 92, 41, 62,  1, 65, 84, 72,
       70, 19, 56, 23, 83, 24, 26, 50, 46,  9, 63, 45, 47, 16, 40, 15, 66,
       59, 17, 86, 38, 67, 21, 36, 44, 33, 75,  4,  0, 28, 49, 89, 57, 20,
       80, 14, 10, 69, 91, 73, 25,  8, 27,  7, 31, 30,  3, 81, 60, 55, 90,
       88,  5, 78, 58, 68, 82, 64,  6, 76, 11, 29, 18, 34, 79, 93, 48, 77,
       37, 54, 22, 42, 43, 51, 71, 39, 13, 61, 32])

In [93]:
batch_size = 16
batches = np.split(file_len.argsort()[::-1], batch_size)

In [97]:
batches

[array([94, 53, 85, 35,  2, 74]),
 array([52, 87, 95, 12, 92, 41]),
 array([62,  1, 65, 84, 72, 70]),
 array([19, 56, 23, 83, 24, 26]),
 array([50, 46,  9, 63, 45, 47]),
 array([16, 40, 15, 66, 59, 17]),
 array([86, 38, 67, 21, 36, 44]),
 array([33, 75,  4,  0, 28, 49]),
 array([89, 57, 20, 80, 14, 10]),
 array([69, 91, 73, 25,  8, 27]),
 array([ 7, 31, 30,  3, 81, 60]),
 array([55, 90, 88,  5, 78, 58]),
 array([68, 82, 64,  6, 76, 11]),
 array([29, 18, 34, 79, 93, 48]),
 array([77, 37, 54, 22, 42, 43]),
 array([51, 71, 39, 13, 61, 32])]

In [98]:
def collate_fn(batch):
    longest = max([len(x) for x in batch])
    s = np.stack([np.pad(x, (0, longest - len(x))) for x in batch])
    return torch.from_numpy(s)

In [100]:
dl = torch.utils.data.DataLoader(dataset=ds, batch_sampler=batches, collate_fn=collate_fn)

In [102]:
for x in dl:
    print(x.shape)

torch.Size([6, 99])
torch.Size([6, 92])
torch.Size([6, 87])
torch.Size([6, 78])
torch.Size([6, 74])
torch.Size([6, 71])
torch.Size([6, 64])
torch.Size([6, 59])
torch.Size([6, 51])
torch.Size([6, 44])
torch.Size([6, 39])
torch.Size([6, 36])
torch.Size([6, 29])
torch.Size([6, 23])
torch.Size([6, 18])
torch.Size([6, 6])
